In [14]:
import pandas as pd

# Load the raw dataset
df = pd.read_csv("../Data/raw_analyst_ratings.csv")
# i have considered MSF for microsoft and FB for meta
# List all unique stock tickers
unique_tickers = df["stock"].dropna().unique()
print(f"Found {len(unique_tickers)} unique tickers:")
print(sorted(unique_tickers))


Found 6204 unique tickers:
['A', 'AA', 'AAC', 'AADR', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAU', 'AAV', 'AAVL', 'AAWW', 'AAXJ', 'AB', 'ABAC', 'ABAX', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCD', 'ABCO', 'ABCW', 'ABDC', 'ABEV', 'ABG', 'ABGB', 'ABIO', 'ABM', 'ABMD', 'ABR', 'ABTL', 'ABX', 'ABY', 'ACAD', 'ACAS', 'ACAT', 'ACC', 'ACCO', 'ACCU', 'ACE', 'ACET', 'ACFC', 'ACFN', 'ACG', 'ACGL', 'ACH', 'ACHC', 'ACHN', 'ACIW', 'ACLS', 'ACM', 'ACMP', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACPW', 'ACRE', 'ACRX', 'ACSF', 'ACST', 'ACT', 'ACTA', 'ACTG', 'ACTS', 'ACU', 'ACUR', 'ACWI', 'ACWV', 'ACWX', 'ACXM', 'ACY', 'ADAT', 'ADBE', 'ADC', 'ADEP', 'ADES', 'ADGE', 'ADHD', 'ADI', 'ADK', 'ADM', 'ADMA', 'ADMP', 'ADMS', 'ADNC', 'ADP', 'ADPT', 'ADRA', 'ADRD', 'ADRE', 'ADRU', 'ADS', 'ADSK', 'ADT', 'ADTN', 'ADVS', 'ADX', 'ADXS', 'ADZ', 'AE', 'AEB', 'AEC', 'AEE', 'AEG', 'AEGN', 'AEGR', 'AEHR', 'AEIS', 'AEL', 'AEM', 'AEO', 'AEPI', 'AER', 'AERI', 'AES', 'AET', 'AETI', 'AEY', 'AEZS', 'AF', 'AFA', 'AFAM', 'AF

In [ ]:
# notebooks/filter_and_save_sentiment.py

import os
import sys
import re
import pandas as pd
import seaborn as sns

sns.set(style="whitegrid")

# Enable inline plotting if in Jupyter
try:
    get_ipython().run_line_magic("matplotlib", "inline")
except Exception:
    pass

# Import custom sentiment extractor
sys.path.append(os.path.abspath(".."))
from src.nlp_utils import extract_sentiment

# --- Configuration ---
news_path = "../Data/raw_analyst_ratings.csv"
output_dir = "../Data/filtered"
os.makedirs(output_dir, exist_ok=True)

# Map raw CSV stock names to correct tickers.
news_to_price_ticker = {
    "AAPL": "AAPL",
    "AMZN": "AMZN",
    "TSLA": "TSLA",
    "MSF": "MSFT",   # 
    "GOOG": "GOOG",
    "FB": "META",    # Rename Facebook to META
    "NVDA": "NVDA"
}

# --- Step 1: Load Raw Data ---
raw_df = pd.read_csv(news_path)
print(f"Original raw shape: {raw_df.shape}")

# Keep only rows with known tickers
df = raw_df[raw_df["stock"].isin(news_to_price_ticker.keys())].copy()
print("Matching tickers in raw data:")
print(df["stock"].value_counts())

# Normalize stock ticker names
df["ticker"] = df["stock"].map(news_to_price_ticker)

# --- Step 2: Custom Date Cleaning Function ---
def clean_date(date_str):
    """
    Convert a date string to YYYY-MM-DD format.
    
    - If the date string contains time and timezone info (e.g., 2020-06-09 11:36:51-04:00),
      it converts it to 'YYYY-MM-DD' (e.g., 2020-06-09).
    - If the date string is already in 'YYYY-MM-DD', it is returned as is.
    - If parsing fails, the original value is returned so that no row is lost.
    """
    try:
        # Try parsing with timezone conversion (assumes valid string)
        dt = pd.to_datetime(date_str, utc=True)
        return dt.strftime('%Y-%m-%d')
    except Exception:
        # If parsing fails, check if it already matches the YYYY-MM-DD pattern
        if isinstance(date_str, str) and re.match(r"^\d{4}-\d{2}-\d{2}$", date_str):
            return date_str
        # Fallback: return the original string to avoid data loss
        return date_str

# Apply the custom date cleaning to the 'date' column.
df["date"] = df["date"].apply(clean_date)

# Optional: Check a few examples of the cleaned date column.
print("Sample cleaned dates:", df["date"].head(10).tolist())

# --- Step 3: Extract Sentiment ---
df = extract_sentiment(df)

# --- Step 4: Save All Data Per Ticker ---
print("\nSaving filtered sentiment-tagged news for each ticker:")
for ticker in news_to_price_ticker.values():
    company_df = df[df["ticker"] == ticker].copy()

    if not company_df.empty:
        output_file = os.path.join(output_dir, f"{ticker}_filtered_news.csv")
        company_df.to_csv(output_file, index=False)
        print(f" {ticker}: {len(company_df)} rows saved → {output_file}")
    else:
        print(f" {ticker}: No records found.")

print("\n Done. All rows are saved with dates in YYYY-MM-DD format (when possible).")


Original raw shape: (1407328, 6)
Matching tickers in raw data:
stock
NVDA    3146
TSLA    1875
GOOG    1199
AAPL     441
FB       380
AMZN     278
MSF       12
Name: count, dtype: int64
Sample cleaned dates: ['2020-06-10', '2020-06-10', '2020-06-10', '2020-06-10', '2020-06-10', '2020-06-10', '2020-06-09', '2020-06-09', '2020-06-09', '2020-06-09']

Saving filtered sentiment-tagged news for each ticker:
 AAPL: 441 rows saved → ../Data/filtered/AAPL_filtered_news.csv
 AMZN: 278 rows saved → ../Data/filtered/AMZN_filtered_news.csv
 TSLA: 1875 rows saved → ../Data/filtered/TSLA_filtered_news.csv
 MSFT: 12 rows saved → ../Data/filtered/MSFT_filtered_news.csv
 GOOG: 1199 rows saved → ../Data/filtered/GOOG_filtered_news.csv
 META: 380 rows saved → ../Data/filtered/META_filtered_news.csv
 NVDA: 3146 rows saved → ../Data/filtered/NVDA_filtered_news.csv

🎉 Done. All rows are saved with dates in YYYY-MM-DD format (when possible).
